In [1]:
import os, json,pickle
import pandas as pd
from langdetect import detect
from collections import Counter

In [2]:
# in data you will store the topic related json files
path = 'data/'

aggregating_fields = ["langMaterial","unitTitle","titleProper","scopeContent","topic"]

dataset = [["id","langMaterial","unitTitle","titleProper","scopeContent","topic","filename"]]

for filename in [x for x in os.listdir(path) if x.endswith()]:
    
    with open(path+filename) as json_file:
        data = json.load(json_file)
    
    for i in range(len(data["response"]["docs"])):
        element = data["response"]["docs"][i]
        
        line = [element["id"]]
        for field in aggregating_fields:
            if field in element:
                line.append(element[field])
            else:
                line.append(" ")
        line.append(filename)
        dataset.append(line)
print ("we have loaded",len(dataset),"entries")

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [3]:
# we remove al columns that do not contain text (where we have less than 3 chars)
selected_dataset = [dataset[0]]+ [x for x in dataset[1:] if len(x[4])>3]

column_names = selected_dataset.pop(0)

# we structure it in a dataframe
df = pd.DataFrame(selected_dataset[1:], columns=column_names)

In [4]:
# as the language is often missing we add it using langdetect (note that the name of the language might diverge, e.g., ger and de)
langs = []

check = set()

for index, row in df.iterrows():
    if len(row["langMaterial"])==3:
        langs.append(row["langMaterial"])
        if row["langMaterial"] not in check:
            print(row["langMaterial"])
            check.add(row["langMaterial"])
        
    else:
        try:
            l = detect(row["unitTitle"] +" "+ row["titleProper"]+" "+ row["scopeContent"])
            row["langMaterial"] = l
            langs.append(row["langMaterial"])
        except Exception as e:
            print (e)
            continue
        

        
from collections import Counter

Counter(langs).most_common()



fre
ger
lav
rus
heb
pol


[('ger', 53316),
 ('fr', 47886),
 ('de', 10983),
 ('fre', 7473),
 ('fi', 2706),
 ('it', 293),
 ('pl', 140),
 ('sv', 52),
 ('lav', 28),
 ('lv', 21),
 ('en', 16),
 ('heb', 14),
 ('rus', 13),
 ('pol', 11),
 ('es', 9),
 ('ca', 8)]

In [6]:
# to fix it we use this dictionary, that you would need to expand if you are dealing with other languages

langs_dict = {"de":"de","ger":"de","fr":"fr","fre":"fr","it":"it","en":"en","sl":"sl","pl":"pl","pol":"pl","fi":"fi"}
df['langMaterial'] = df['langMaterial'].map(langs_dict)


In [7]:
# finally we save the pickle file containing the dataframe
with open('dataset.pickle', 'wb') as f:
        pickle.dump(df, f)